<a href="https://colab.research.google.com/github/xyzalanix/aiartathon2.0/blob/main/1_pi_GAN_3D_AwareImageSynthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# π-GAN: Periodic Implicit Generative Adversarial Networks for 3D-Aware Image Synthesis

Eric Ryan Chan*, Marco Monteiro*, Petr Kellnhofer, Jiajun Wu, Gordon Wetzstein
*denotes equal contribution

This is the official implementation of the paper "π-GAN: Periodic Implicit Generative Adversarial Networks for 3D-Aware Image Synthesis".

π-GAN is a novel generative model for high-quality 3D aware image synthesis.

# Setup

Run on colab runtime

In [ ]:
!git clone https://github.com/marcoamonteiro/pi-GAN
%cd /content/pi-GAN/
!pip install -r requirements.txt

Or on gdrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/pi-GAN"):
    %cd "/content/drive/MyDrive/pi-GAN"
    !pip install -r requirements.txt
else:
    #install script
    %cd "/content/drive/MyDrive"
    !git clone https://github.com/marcoamonteiro/pi-GAN
    %cd /content/drive/MyDrive/pi-GAN
    !pip install -r requirements.txt

# Training

Point to your dataset and its parameters in **curriculums.py**

In [ ]:
%cd /content/drive/MyDrive/pi-GAN

!CUDA_VISIBLE_DEVICES=0 python3 train.py --curriculum camelgrief --output_dir camelgrief-CLIPOG-Test \
#--load_dir=./mineralTest

/content/drive/MyDrive/pi-GAN
Namespace(curriculum='camelgrief', eval_freq=0, load_dir='', model_save_interval=5000, n_epochs=3000, output_dir='camelgrief-CLIPOG-Test', port='12355', sample_interval=200, set_step=None)
Total progress:   0% 0/3000 [00:00<?, ?it/s]
0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessa

# Generation

In [ ]:
#@title ##Stills
size = 128 #@param
generator = './mineralTest/09--13:10--generator.pth' #@param ['./mineralTest/generator.pth', './mineralTest/08--21:33--generator.pth', './mineralTest/09--13:10--generator.pth','./memes/generator.pth','./flowersTest/generator.pth']
seeds = "101 102 103 104 105 106 107 108 109 110" #@param {type:"string"}
output_dir = "/content/" #@param {type:"string"}

!python render_multiview_images.py $generator --image_size $size --curriculum CelebA --ray_step_multiplier 1 \
--seeds $seeds \
--output_dir $output_dir

100% 10/10 [00:00<00:00, 11.94it/s]


https://numpy.org/doc/stable/reference/routines.math.html

In [ ]:
#@title Collect trash, empty cache
import gc
import torch

torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
#@title ##Interpolations
size =  128#@param
generator = './mineralTest/08--21:33--generator.pth' #@param ['./mineralTest/generator.pth', './mineralTest/08--21:33--generator.pth', './mineralTest/09--13:10--generator.pth','./memes/generator.pth','./flowersTest/generator.pth']
trajectory = "orbit" #@param ['front','orbit']
seeds = "121" #@param {type:"string"}
psi =  0.8#@param
num_frames =  60#@param
output_dir = "/content/" #@param {type:"string"}


!python render_video_interpolation.py $generator --psi $psi --image_size $size --curriculum CelebA --ray_step_multiplier 1 \
--seeds $seeds \
--output_dir $output_dir  --num_frames $num_frames --trajectory $trajectory

['121']
  0% 0/60 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/PIL/Image.py", line 2772, in fromarray
    mode, rawmode = _fromarray_typemap[typekey]
KeyError: ((1, 1, 128), '|u1')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "render_video_interpolation.py", line 132, in <module>
    frames.append(tensor_to_PIL(frame))
  File "render_video_interpolation.py", line 65, in tensor_to_PIL
    return Image.fromarray(img.mul(255).add_(0.5).clamp_(0, 255).permute(0, 2, 1).to('cpu', torch.uint8).numpy())
  File "/usr/local/lib/python3.7/dist-packages/PIL/Image.py", line 2774, in fromarray
    raise TypeError("Cannot handle this data type: %s, %s" % typekey) from e
TypeError: Cannot handle this data type: (1, 1, 128), |u1


In [ ]:
#Videos (thanks Kaitlin Schaer!)
def show_local_mp4_video(file_name, width=300, height=300):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('/content/interp.mp4', width=300, height=300)

In [ ]:
!python extract_shapes.py ./mineralTest/generator.pth --seed 1